In [1]:
import torch.nn as nn
import torch
import importlib
import sinn.dataset.dataset
import sinn.simulation.simulation
import sinn.train.train
import sinn.graph.graph
sinn.dataset.dataset = importlib.reload(sinn.dataset.dataset)
sinn.train.train = importlib.reload(sinn.train.train)
sinn.graph.graph = importlib.reload(sinn.graph.graph)
sinn.simulation.simulation = importlib.reload(sinn.simulation.simulation)


from MDAnalysis.coordinates.GSD import GSDReader
from MDAnalysis import Universe

from sinn.dataset.dataset import FilteredAtomsDataset, collate_noise
from sinn.model.model import SchNet, Alignn
from sinn.train.train import run_epoch, SimulatedNoiseRegressionEval

/Users/johnmarquardt/anaconda3/envs/colloids/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-08 18:11:03,330	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-08 18:11:03,414	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
n_atoms = 2
spg = ('225',)
categorical_filter = ([True],['spg_number'],[spg])

batch_size = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

k = 20
pre_eval_func = SimulatedNoiseRegressionEval(k = k)

dataset = Universe('./test_traj/trajectory_LS4_FP0.5_RN105_BL10_DL5.3_Th3P4.gsd')

dataset = FilteredAtomsDataset(source = dataset,
                               transform=pre_eval_func,
                               target = 'target').dataset

Taking dataset with size (7555, 3)
Dataset reduced to size (7555, 3)


In [3]:
dataset[0]

(Graph(num_nodes=6750, num_edges=135000,
       ndata_schemes={'z': Scheme(shape=(), dtype=torch.int32), '_ID': Scheme(shape=(), dtype=torch.int64)}
       edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32), 'd': Scheme(shape=(), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}),
 tensor(0.))

In [4]:
model_name = 'SchNet-AtomNoise-Spg225'
model_path = 'models/24-05-29/'
model = SchNet(num_classes=1, num_layers=3, hidden_features=64, radial_features=128)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
run_epoch(model, dataset, loss_func, optimizer, batch_size, device, model_name, model_path)

  0%|          | 0/7555 [00:00<?, ?it/s]/Users/johnmarquardt/anaconda3/envs/colloids/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 24%|██▎       | 1776/7555 [5:59:22<19:29:21, 12.14s/it]  


KeyboardInterrupt: 